# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770551179004                   -0.53    9.0         
  2   -2.772137674712       -2.80       -1.29    1.0    172ms
  3   -2.772169829649       -4.49       -2.71    1.0    240ms
  4   -2.772170718882       -6.05       -3.73    2.0    217ms
  5   -2.772170722755       -8.41       -4.23    1.0    186ms
  6   -2.772170722994       -9.62       -5.46    1.0    226ms
  7   -2.772170723013      -10.71       -5.39    2.0    221ms
  8   -2.772170723015      -11.80       -6.24    1.0    196ms
  9   -2.772170723015      -13.41       -6.79    1.0    249ms
 10   -2.772170723015   +  -14.27       -7.17    2.0    244ms
 11   -2.772170723015      -14.12       -7.93    1.0    218ms
 12   -2.772170723015   +  -15.05       -8.11    1.0    236ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735581202054613

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770707622056                   -0.53    8.0         
  2   -2.772055260826       -2.87       -1.31    1.0    224ms
  3   -2.772083075044       -4.56       -2.59    1.0    177ms
  4   -2.772083373594       -6.52       -3.51    1.0    220ms
  5   -2.772083415644       -7.38       -3.88    2.0    208ms
  6   -2.772083417761       -8.67       -5.28    1.0    214ms
  7   -2.772083417805      -10.35       -5.20    3.0    244ms
  8   -2.772083417810      -11.30       -5.74    1.0    210ms
  9   -2.772083417811      -12.71       -6.67    1.0    209ms
 10   -2.772083417811      -14.35       -7.35    2.0    258ms
 11   -2.772083417811      -14.45       -7.12    2.0    251ms
 12   -2.772083417811      -13.85       -7.94    1.0    217ms
 13   -2.772083417811   +  -13.80       -8.97    2.0    254ms

Polarizability via ForwardDiff:       1.7725349692472743
Polarizabilit